In [29]:
from __future__ import print_function
import os
import shutil
import sys
import PIL
import glob
from matplotlib.pyplot import imshow
from PIL import  Image, ImageDraw
import matplotlib.pyplot as plt
import numpy as np
import cPickle
%matplotlib notebook


In [30]:
#step 1 : read in bundle file
#step 2 : get the "0-cam" rate
#step 3 : if rate < thresh(or other heuristic):  we claim it is not good enough
#step 4 : first compute all the world-coordinate position for all valid segments (also for invalid bundle)
#step 5 : We can save them in a matrix-like
#step 6 : Store in file

In [33]:
def parseBundlerFile(fname):
    f = open(fname,'r')
    count = 0

    for lines in f.readlines():
        count = count + 1
        if count == 1: 
            # this is the title file
            continue
        elif count == 2:
            # second line is num cameras & num keypoints
            tmp = lines.split()
            num_cam = int(tmp[0])
            num_point = int(tmp[1])
            # the reading limit for camera information, each cam is described by 5 lines
            cam_limit = count + 5 * num_cam
            w, h = 3, 5*num_cam
            # camera information output
            CAM = [[0 for x in range(w)] for y in range(h)] 
            w_p,h_p = 3,3*num_point
            p_flag = 0
            PNT = [[0 for x in range(w_p)] for y in range(h_p)]
        elif count <= cam_limit:
            # those 0,1,2 has different meaning for different lines but all lines has 3 numbers
            CAM[count-3][0] = float(lines.split()[0])
            CAM[count-3][1] = float(lines.split()[1])
            CAM[count-3][2] = float(lines.split()[2])
        else:
            p_flag = p_flag + 1
            if p_flag == 1:
                #print count,h
                # 3D position
                for pos in range(0,3):
                    PNT[count-h-3][pos] = float(lines.split()[pos])
            elif p_flag == 2:
                #print count,h
                # RGB color of this keypoint
                for rgb_p in range(0,3):
                    PNT[count-h-3][rgb_p] = float(lines.split()[rgb_p])
            elif p_flag == 3:
                p_flag = 0
                sp=lines.split()
                assert(len(sp)==(4*int(sp[0])+1))
                for view_p in range(4*int(sp[0])):
                    sp[view_p+1]=float(sp[view_p+1])
                PNT[count-h-3]=sp[1:]
    f.close()
    return CAM, PNT

def parseCam(cam):
    # parse the camera into better format
    cam=np.asarray(cam)
    out=[]
    num_cam=int(len(cam)/5)
    for i in range(num_cam):
        this_cam={}
        subcam=cam[i*5:(i+1)*5]
        this_cam["focal_len"]=subcam[0][0]
        this_cam["distort_coeff"]=subcam[0][1:]
        this_cam["R"]=np.matrix(subcam[1:4])
        this_cam["t"]=np.matrix(subcam[4]).T
        out.append(this_cam)
    return out

def parseKeypoints(pnt):
    num_keypoint=int(len(pnt)/3)
    out=[]
    for i in range(num_keypoint):
        this_point={}
        subpnt=pnt[i*3:(i+1)*3]
        this_point["position"]=np.matrix(subpnt[0]).T
        this_point["color"]=np.asarray(subpnt[1])
        # parse the occur of this keypoint in all cameras
        view_list=[]
        for j in range(int(len(subpnt[2])/4)):
            this_cam={}
            subsubpnt=subpnt[2][j*4:(j+1)*4]
            this_cam["camera_index"]=int(subsubpnt[0])
            this_cam["sift_index"]=int(subsubpnt[1])
            this_cam["position"]=np.matrix(subsubpnt[2:]).T
            view_list.append(this_cam)
        this_point["view_list"]=view_list
        
        out.append(this_point)
    return out
def getdata(filename):
    cam, kp = parseBundlerFile(filename)
    cam = parseCam(cam)
    kp = parseKeypoints(kp)
    point_num = len(kp)
    total_num = len(cam)
    zero_num = 0
    pos_matrix = [[0 for x in range(total_num)] for y in range(total_num)] 
    return cam, kp, point_num, total_num, zero_num, pos_matrix
def position(cam_1,cam_2):
    cam_2_world = -cam_2['R'].T*cam_2['t']
    position = cam_1['R']*cam_2_world + cam_1['t']
    return position
def data_update(total_num, cam, zero_num, filename,pos_matrix):
    dist = []
    for i in xrange(0,total_num):
        if  np.count_nonzero(cam[i]['R'])==0  and np.count_nonzero( cam[i]['t'] ) == 0:
            zero_num = zero_num + 1
        if i !=total_num-1:
            dist.append(np.linalg.norm(cam[i+1]['t']-cam[i]['t']))    
        # compute the position
        for j in xrange(i+1,total_num):
            pos_matrix[i][j] = position(cam[i],cam[j])
    tmp = filename.split('/')
    name = tmp[1]+'/'+tmp[2]
    return dist, pos_matrix, name, zero_num


In [86]:
# read file in a loop
file_num = 0;
data = []
all_file = sorted(glob.iglob(os.path.join('./', '*','seg*','bundle', 'bundle.out')))
for filename in all_file:
    cam,kp,point_num, total_num,zero_num, pos_matrix = getdata(filename)
    file_num = file_num + 1;
    
    
    # check camera
    dist, pos_matrix, name, zero_num=data_update(total_num, cam, zero_num, filename,pos_matrix)
    
    #check dis
    avg  = sum(dist)/len(dist)
    flag = [ x>5*avg for x in dist]
    index = [i for i, j in enumerate(flag) if j == True]
    
    
    
    this_data = {"name":name,"cam_num":total_num,"egomotion":pos_matrix,"valid":0,"ignore_list":index}
    
    
    rate = float(zero_num)/float(total_num)
    if rate <=0.05 and point_num > 1500:
        # ok the data seems good here! this is tight! We can relax this later.
        this_data["valid"] = 1
    data.append(this_data)
    print(file_num)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [88]:
cPickle.dump(data, open("data.dat", "w"))

obj = cPickle.load(open("data.dat", "r"))
print(len(obj))

859


859
